# RNN


In [1]:
import numpy as np
import torch

# Setando sementes
np.random.seed(42)
torch.manual_seed(42)

# Configurando tipo das variáveis
dtype = torch.float32
torch.set_default_dtype(dtype)

# Definindo nome do modelo
model_name = "RNN"


## Lendo dados e preparando tensores


In [2]:
import pandas as pd

df = pd.read_csv("../export/dados.csv", index_col=0)

ciclos = df.index.values
t_feed = df["t_feed"].values
t_rinse = df["t_rinse"].values
t_blow = df["t_blow"].values
t_purge = df["t_purge"].values

purity_H2 = df["purity_H2"].values
H2_CO_ratio = df["H2_CO_ratio"].values
purity_CO2 = df["purity_CO2"].values
recovery_CO2 = df["recovery_CO2"].values
productivity = df["productivity"].values


In [3]:
y = np.array(
    [
        purity_H2,
        H2_CO_ratio,
        purity_CO2,
        recovery_CO2,
        productivity,
    ]
).T
u = np.array([t_feed, t_rinse, t_blow, t_purge]).T


In [4]:
from lib.models import DataInfo
from lib.utils import prepare_sequences

# Concatena entradas (y_k-1, u_k)
X = np.hstack([y[:-1], u[1:]])  # (N-1, 9)
Y = y[1:]  # (N-1, 5)

# Converte para tensores
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.float32)

dataInfo: DataInfo = {
    "in_min": torch.min(X, dim=0).values,
    "in_max": torch.max(X, dim=0).values,
    "out_min": torch.min(Y, dim=0).values,
    "out_max": torch.max(Y, dim=0).values,
}

# Preparando sequences
input_sequences = X.T  # (9, N-1)
target_sequences = Y.T  # (5, N-1)

history_size = 2
X_seq, Y_seq = prepare_sequences(
    input_sequences, target_sequences, history_size=history_size, overlap=True
)

print("X_seq:", X_seq.shape)
print("Y_seq:", Y_seq.shape)

# Divide em treino e validação
N = len(X_seq)
N_train = int(0.8 * N)
N_val = N - N_train

X_train = X_seq[:N_train]
Y_train = Y_seq[:N_train]

X_val = X_seq[N_train:]
Y_val = Y_seq[N_train:]


X_seq: torch.Size([97, 2, 9])
Y_seq: torch.Size([97, 5])


## Criando e Treinando a Rede Neural


In [5]:
from lib.models import RNN

input_size = X.shape[1]
hidden_size = 16
output_size = Y.shape[1]

model = RNN(
    input_size=input_size,
    hidden_size=hidden_size,
    output_size=output_size,
    dataInfo=dataInfo,
)


In [6]:
import os

from lib.plot import plot_loss

MODEL_PATH = f"../export/{model_name}.pt"

if os.path.exists(MODEL_PATH):
    print("📦 Modelo encontrado. Carregando...")
    model.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"))
    model.eval()
else:
    print("🚀 Modelo não encontrado. Treinando agora...")

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    history, best_epoch = model.fit(
        X_train=X_train,
        Y_train=Y_train,
        X_val=X_val,
        Y_val=Y_val,
        optimizer=optimizer,
        epochs=10000,
    )

    plot_loss(
        history["train_loss"][: best_epoch + 1],
        history["val_loss"][: best_epoch + 1],
        model_name,
    )

    # Salvar modelo após treino
    torch.save(model.state_dict(), MODEL_PATH)
    print(f"💾 Modelo salvo em {MODEL_PATH}")


📦 Modelo encontrado. Carregando...


## Simulando


In [7]:
from lib.utils import denormalize, normalize


@torch.inference_mode()
def infer_RNN(model, y, u, history_size: int):
    model.eval()
    n_samples, n_outputs = y.shape

    # saída predita
    y_pred = torch.zeros_like(y)

    # normalizar y e u inteiros
    y_norm = normalize(y, model.in_min[:5], model.in_max[:5])
    u_norm = normalize(u, model.in_min[5:], model.in_max[5:])

    # copiar valores iniciais (y[0], y[1] se history_size=2)
    y_pred[:history_size] = y_norm[:history_size]

    for k in range(history_size, n_samples):
        # preencher seq com valores passados
        seq_y = y_pred[k - history_size : k]  # (history_size, 5)
        seq_u = u_norm[k - history_size + 1 : k + 1]  # (history_size, 4)

        seq = torch.cat([seq_y, seq_u], dim=1).reshape(1, history_size, -1)

        y_pred[k] = model(seq, disable_norm=True)[0]  # (5,)

    # desnormalizar
    y_denorm = denormalize(
        y_pred,
        model.out_min[:5],
        model.out_max[:5],
    )

    return y_denorm


In [8]:
from lib.plot import plot_comparison
from lib.utils import benchmark

y = torch.tensor(y, dtype=torch.float32)
u = torch.tensor(u, dtype=torch.float32)

y_nn = infer_RNN(model, y, u, history_size)

mse = torch.mean((y - y_nn) ** 2).numpy()
print(f"MSE {model_name}:", mse)

plot_comparison(ciclos, y, y_nn.numpy(), model_name)

benchmark(infer_RNN, model_name, model, y, u, history_size)


MSE RNN: 2.454888
Benchmark file already exists. Skipping benchmark.
